In [2]:
import numpy as np
import pandas as pd

In [3]:
file_path = "us-counties-covid19.csv"
df = pd.read_csv(file_path,dtype={'fips':'object'})
df.dtypes

date      object
county    object
state     object
fips      object
cases      int64
deaths     int64
dtype: object

In [4]:
df

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0
...,...,...,...,...,...,...
366694,2020-07-24,Sweetwater,Wyoming,56037,210,2
366695,2020-07-24,Teton,Wyoming,56039,264,1
366696,2020-07-24,Uinta,Wyoming,56041,232,0
366697,2020-07-24,Washakie,Wyoming,56043,44,5


In [5]:
df["date"] = pd.to_datetime(df['date'])
df

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0
...,...,...,...,...,...,...
366694,2020-07-24,Sweetwater,Wyoming,56037,210,2
366695,2020-07-24,Teton,Wyoming,56039,264,1
366696,2020-07-24,Uinta,Wyoming,56041,232,0
366697,2020-07-24,Washakie,Wyoming,56043,44,5


In [6]:
df.loc[df["county"] == "New York City", "fips"] = df.loc[df["county"] == "New York City", "fips"].fillna('36061')
df[df["county"] == "New York City"]

,date,county,state,fips,cases,deaths
416,2020-03-01,New York City,New York,36061,1,0
448,2020-03-02,New York City,New York,36061,1,0
482,2020-03-03,New York City,New York,36061,2,0
518,2020-03-04,New York City,New York,36061,2,0
565,2020-03-05,New York City,New York,36061,4,0
...,...,...,...,...,...,...
352513,2020-07-20,New York City,New York,36061,226388,22882
355717,2020-07-21,New York City,New York,36061,226779,22895
358922,2020-07-22,New York City,New York,36061,227130,22899
362129,2020-07-23,New York City,New York,36061,227517,22934


In [7]:
# df.set_index('date').resample('W')["cases"].sum()
# df.set_index('date').groupby('fips')['cases'].resample("W").sum()

In [8]:
df_g = df.groupby(['fips', pd.Grouper(key='date', freq='W-SUN'), 'county', 'state'])[['cases', 'deaths']].last()
df_g

cases  deaths
fips  date       county     state                        
01001 2020-03-29 Autauga    Alabama             6       0
      2020-04-05 Autauga    Alabama            12       0
      2020-04-12 Autauga    Alabama            19       1
      2020-04-19 Autauga    Alabama            27       2
      2020-04-26 Autauga    Alabama            37       2
...                                           ...     ...
78030 2020-06-28 St. Thomas Virgin Islands     40       0
      2020-07-05 St. Thomas Virgin Islands     45       0
      2020-07-12 St. Thomas Virgin Islands     62       0
      2020-07-19 St. Thomas Virgin Islands     90       0
      2020-07-26 St. Thomas Virgin Islands    114       3

[54136 rows x 2 columns]

In [9]:
# df_g = df_g.reset_index().set_index('date').reset_index()
df_g = df_g.reset_index().sort_values(by='date')
df_g['week'] = df_g['date'].dt.week
df_g = df_g.set_index('week')
df_g

,fips,date,county,state,cases,deaths
week,,,,,,
4,06059,2020-01-26,Orange,California,1,0
4,06037,2020-01-26,Los Angeles,California,1,0
4,17031,2020-01-26,Cook,Illinois,1,0
4,53061,2020-01-26,Snohomish,Washington,1,0
4,04013,2020-01-26,Maricopa,Arizona,1,0
...,...,...,...,...,...,...
30,29045,2020-07-26,Clark,Missouri,2,0
30,29043,2020-07-26,Christian,Missouri,211,1
30,53007,2020-07-26,Chelan,Washington,859,10


In [10]:
# df.index.to_series()

In [11]:
# df_g = df_g.rename(columns={'fips':'id','cases':'rate'}) #.drop(columns=['date'])

In [12]:
df_g.to_csv("./us-counties-covid19-weekly.csv", index=True)

In [18]:
df_us = df.groupby(['date'])[['cases','deaths']].sum().reset_index()
df_us.to_csv("./us-total-covid19-daily.csv", index=False)

In [12]:
df_test = df_g.groupby(['fips'])['cases'].max().reset_index().rename(columns={'fips':'fips'})
df_test.to_csv("./us-counties-covid19-test.csv", index=False)

In [13]:
df_test

,fips,cases
0,01001,921
1,01003,2513
2,01005,539
3,01007,303
4,01009,615
...,...,...
3177,72151,69
3178,72153,167
3179,78010,223
3180,78020,15


In [14]:
df_g[df_g["fips"]=='36061']

,fips,date,county,state,cases,deaths
week,,,,,,
9,36061,2020-03-01,New York City,New York,1,0
10,36061,2020-03-08,New York City,New York,14,0
11,36061,2020-03-15,New York City,New York,330,5
12,36061,2020-03-22,New York City,New York,9065,119
13,36061,2020-03-29,New York City,New York,33983,991
14,36061,2020-04-05,New York City,New York,68726,3646
15,36061,2020-04-12,New York City,New York,105939,9448
16,36061,2020-04-19,New York City,New York,138370,13632
17,36061,2020-04-26,New York City,New York,163106,16486


In [15]:
df[df["fips"].isnull()].sort_values(by='state')

,date,county,state,fips,cases,deaths
299815,2020-07-04,Unknown,Alaska,NaN,6,0
190788,2020-05-30,Unknown,Alaska,NaN,1,0
284041,2020-06-29,Unknown,Alaska,NaN,11,0
274595,2020-06-26,Unknown,Alaska,NaN,10,0
237117,2020-06-14,Unknown,Alaska,NaN,3,0
...,...,...,...,...,...,...
328260,2020-07-12,Unknown,Wisconsin,NaN,3317,7
45795,2020-04-09,Unknown,Wyoming,NaN,5,0
43197,2020-04-08,Unknown,Wyoming,NaN,30,0
59163,2020-04-14,Unknown,Wyoming,NaN,1,0
